In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
from sqlalchemy import create_engine, text

PostgreSQL_HOST = '127.0.0.1'
PostgreSQL_PORT = '5432'
PostgreSQL_USER = 'postgres'
PostgreSQL_PASSWORD = '123456'
PostgreSQL_DB = 'sql_advanced'

engine = create_engine('postgresql+psycopg2://%s:%s@%s:%s/%s'
                           % (PostgreSQL_USER, PostgreSQL_PASSWORD, PostgreSQL_HOST, PostgreSQL_PORT, PostgreSQL_DB))

In [2]:
# 用于获取可重排列的 SQL 语句
query_1 = """
SELECT P1.name AS name_1, P2.name AS name_2
 FROM Products P1, Products P2;
"""

cross_df = pd.read_sql_query(query_1, engine) 
cross_df

,name_1,name_2
0,苹果,苹果
1,苹果,橘子
2,苹果,香蕉
3,橘子,苹果
4,橘子,橘子
5,橘子,香蕉
6,香蕉,苹果
7,香蕉,橘子
8,香蕉,香蕉


In [7]:
query  = """select * from Products"""
data = pd.read_sql_query(query, engine) 

data

df1 = data.copy()  
df2 = data.copy() 
result = pd.merge(df1, df2, on='name', suffixes=('_1', '_2'))  
result

,name,price_1,price_2
0,苹果,50,50
1,橘子,100,100
2,香蕉,80,80


In [25]:
query  = """select * from Products"""
Products = pd.read_sql_query(query, engine) 

# 将name列转换为一个DataFrame
name_df = Products["name"].to_frame()

# 创建一个公共的键列，值都为0
name_df["key"] = 0

# 用merge函数进行交叉连接，on参数指定公共的键列，how参数指定连接方式为outer
cross_join = name_df.merge(name_df, on="key", how="outer").rename(columns={"name_x": "name_1", "name_y": "name_2"})

# 删除结果表中的键列
cross_join = cross_join[['name_1','name_2']]

# 查看结果表
cross_join

,name_1,name_2
0,苹果,苹果
1,苹果,橘子
2,苹果,香蕉
3,橘子,苹果
4,橘子,橘子
5,橘子,香蕉
6,香蕉,苹果
7,香蕉,橘子
8,香蕉,香蕉


In [29]:
query  = """select * from Products"""
Products = pd.read_sql_query(query, engine) 

# 将name列转换为一个DataFrame
name_df = Products["name"].to_frame()

# 创建一个公共的键列，值都为0
name_df["key"] = 0

# 用merge函数进行交叉连接，on参数指定公共的键列，how参数指定连接方式为outer
cross_join = name_df.merge(name_df, on="key", how="outer").rename(columns={"name_x": "name_1", "name_y": "name_2"})

# 用query函数筛选出结果表中name_1和name_2不相等的行
# inner_join = cross_join.query("name_1 != name_2")

cross_join.drop_duplicates(inplace=True)

# 查看结果表
inner_join


,name_1,key,name_2
1,苹果,0,橘子
2,苹果,0,香蕉
3,橘子,0,苹果
5,橘子,0,香蕉
6,香蕉,0,苹果
7,香蕉,0,橘子
